<a href="https://colab.research.google.com/github/alejogiley/Hack7/blob/master/Unqualified_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Anaconda

In [3]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local -c omnia --no-update-deps pdbfixer=1.4
!conda install -q -y --prefix /usr/local -c conda-forge --no-update-deps xgboost=0.6a2
!conda install -q -y --prefix /usr/local -c rdkit --no-update-deps rdkit=2017.09.1
!conda install -q -y --prefix /usr/local -c deepchem --no-update-deps  deepchem-gpu=2.1.0

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')
#ignore warning about PYTHONPATH

--2019-08-17 02:35:58--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M  76.1MB/s    in 7.4s    

2019-08-17 02:36:05 (74.4 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: bzip2-1.0.6-h9a117

In [0]:
import os
import pandas as pd
import numpy as np

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Stats
from scipy.stats import skew

#Importing scikit-learn
import sklearn as sk
#Fill---
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, RobustScaler 
from sklearn.preprocessing import Imputer
#Normalizing
from sklearn import preprocessing
#Data split
from sklearn.model_selection import train_test_split 
#Random Forest packages
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV
#Evaluation package
from sklearn.metrics import accuracy_score

#Importing XG_BOOST packages
import xgboost as xg
from xgboost import plot_importance

#Importing Neural Network Packages
from sklearn.neural_network import MLPClassifier

#Importing rdkit packages
from rdkit import RDConfig
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.ML.Descriptors import MoleculeDescriptors

#Importing DeepChem
import deepchem as dc

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Load Data
---------------

Antibiotics literature dataset

In [7]:
# Load Dataset
df = pd.read_csv('/content/drive/My Drive/Hack 7/Gyrase/AZ_Pyrrolamides_4OE.csv', 
                 encoding='unicode_escape') 
df.head()

,External_ID,Molecule SMILES,First Sample Reg. Date,SAU Gyr IC50 (µM),SAU Gyr Ki (µM),Sau516 MIC (µg/ml),Sau516 MIC (µM),Mean Human Prot binding (% free),Mean LogD,Determination of pKa HA1,Determination of pKa HA2,Determination of pKa B1,Determination of pKa B2,Sirius pKa
0,AZ1001,c1cc(c(nc1)N2CCC(CC2)NC(=O)c3cc(c([nH]3)Cl)Cl)[N+](=O)[O-],6/10/2002,>400,NaN,>64,NaN,NaN,NV,NaN,NaN,NaN,NaN,NaN
1,AZ1002,CC(C)c1c(cc([nH]1)C(=O)NC2CCN(CC2)c3c(cccn3)[N+](=O)[O-])Br,8/16/2002,>50,NaN,>64,NaN,NaN,>4,NaN,NaN,NaN,NaN,NaN
2,AZ1003,CCOC(=O)c1cc(nc(c1C#N)N2CCC(CC2)NC(=O)c3cc(c([nH]3)C)Br)C,8/16/2002,7.14,NaN,>64,NaN,NaN,>3.66,NaN,NaN,NaN,NaN,NaN
3,AZ1004,Cc1c(cc([nH]1)C(=O)NC2CCN(CC2)c3c(cccn3)C#N)Br,8/16/2002,9.93,NaN,>64,NaN,NaN,>3.13,NaN,NaN,NaN,NaN,NaN
4,AZ1005,Cc1c(cc([nH]1)C(=O)NC2CCN(CC2)c3ccc4ccccc4n3)Br,8/16/2002,3.31,NaN,>64,NaN,NaN,>4.07,NaN,NaN,NaN,NaN,NaN


SMILES
---------

Check SMILES are correct.


In [29]:
# Get Smiles
pp = pd.DataFrame()
pp[['ID','Smiles']] = df[['External_ID','Molecule SMILES']]
pp['ProblematicSmiles'] = pd.Series()

# Flag bad smiles
def smile2mol(x):
  m = Chem.MolFromSmiles(x)
  if m is None:
    return None
  else:
    return m

# Get Mol objects 
pp['Molecule'] = pp['Smiles'].apply(lambda x: smile2mol(x))

# find index of problematic smiles
ps = [ k for k,v in enumerate(pp['Molecule']) if v is None ]

# Fix problematic smiles ??
if len(ps) > 0:
  
  print("Having problem getting structures for these smiles:")
  pp['ProblematicSmiles'] = pp['Smiles'].loc[ps]
  
  for i in range(len(ps)):
    print("index: %i" % ps[i], pp['ProblematicSmiles'].loc[ps[i]])
  
  print("Replacing character \'\\2\' with \'2\'")
  pp['Smiles'] = pp['Smiles'].str.replace('\2','2', regex=True)
  
  print("Regenerating molecules")
  pp['Molecule'] = pp['Smiles'].apply(lambda x: smile2mol(x))
  
  if False in pp['Molecule'].isnull():
    print("No problema compadre")

RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3ncc(s3)C=O)Cl)Cl'
RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)OCC=C)C(=O)NCCOC)Cl)Cl'
RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)C(=O)NCCOC)Cl)Cl'
RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)OC)C(=O)NC)Cl)Cl'
RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)C(=O)NC)Cl)Cl'
RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)c4nccn4COC)Cl)Cl'
RDKit ERROR: [03:26:39] SMILES Parse Error: syntax error for input: 'Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)c4[nH]ccn4)Cl)Cl'
RDKit ERROR: [03:26:39] SMILE

Having problem getting structures for these smiles:
index: 698 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3ncc(s3)C=O)Cl)Cl
index: 701 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)OCC=C)C(=O)NCCOC)Cl)Cl
index: 702 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)C(=O)NCCOC)Cl)Cl
index: 707 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)OC)C(=O)NC)Cl)Cl
index: 708 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)C(=O)NC)Cl)Cl
index: 714 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)c4nccn4COC)Cl)Cl
index: 715 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)c4[nH]ccn4)Cl)Cl
index: 717 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)C(=O)N)Cl)Cl
index: 733 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)OC)c4nccn4CCOC)Cl)Cl
index: 736 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O)c4nccn4CCOC)Cl)Cl
index: 741 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)OC)c4ncnn4C)Cl)Cl
index: 744 Cc1c(c(c([nH]1)C(=O)NCCCN(C/C2=N\OC)c3nc(c(s3)C(=O)O

Descriptors
------------------

RDKit molecular descriptors

In [30]:
# Descriptors
descriptors = list(np.array(Descriptors._descList)[:,0])
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors) 

# Make is easier to identify problematic molecules later
# e.g infity descriptor values
def computeDescriptors(mol, calculator):
  res = np.array(calculator.CalcDescriptors(mol))
  if not np.all(np.isfinite(res)):
    return None 
  return res 

# Selecting only non-null molecules 
pp = pp[ pp["Molecule"].notnull() ]

# Mapping descriptors with molecules
pp['Descriptors'] = pp['Molecule'].map(lambda x: computeDescriptors(x, calculator)) 

# Flags problems
ps = pp["Descriptors"].isnull()
print("{} molecules failed to get descriptors".format(ps.sum())) 

# Rename columns
des = pp["Descriptors"].apply(pd.Series)
des = des.rename(columns = lambda x : descriptors[x])

0 molecules failed to get descriptors


In [32]:
# Concatenating main and descriptor dataframes
dataset = pd.concat([pp[["ID", "Smiles"]], df["SAU Gyr IC50 (µM)"], des], axis=1)
dataset = dataset.dropna(how ='any')
dataset[descriptors].head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,...,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.198481,-0.435597,12.198481,0.013283,0.623001,384.223,369.103,383.055195,132.0,0.0,0.311075,-0.350740,0.350740,0.311075,1.400000,2.120000,2.720000,1.606288,782.714558,17.974327,13.037493,14.549351,11.952397,7.646576,8.402505,5.682961,6.468125,4.062271,4.755185,2.799838,3.112547,-2.32,5.332598e+05,17.501340,7.283006,3.853433,152.831853,15.200677,10.847038,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.517532,-0.411106,12.517532,0.011515,0.550709,436.310,414.134,435.090602,144.0,0.0,0.311075,-0.353241,0.353241,0.311075,1.444444,2.148148,2.740741,1.607439,843.272683,19.551677,15.236879,16.822875,12.863081,8.900969,9.693967,6.937354,7.791691,4.712163,5.368251,3.261192,4.047073,-2.42,1.314876e+06,19.344590,8.077319,4.292290,165.187695,15.200677,5.693928,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.472623,-0.518193,12.472623,0.024077,0.644188,474.359,450.167,473.106252,158.0,0.0,0.339176,-0.462356,0.462356,0.339176,1.366667,2.066667,2.633333,1.707098,983.301265,21.836134,17.211935,18.797931,14.332938,9.833892,10.626890,7.232812,8.087149,5.061381,5.885049,3.519471,3.847515,-2.86,5.145049e+06,21.839461,9.303515,4.648309,183.297193,19.937540,23.144464,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.298197,-0.085370,12.298197,0.085370,0.847033,388.269,370.125,387.069472,124.0,0.0,0.267458,-0.355369,0.355369,0.267458,1.500000,2.250000,2.875000,1.591350,767.018857,17.104084,13.343032,14.929029,11.579719,7.842445,8.635443,5.877107,6.731444,4.232941,5.056609,2.890092,3.218135,-2.33,3.779222e+05,16.525072,7.054447,3.436063,148.562357,15.200677,17.581012,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.387258,-0.040667,12.387258,0.040667,0.681761,413.319,392.151,412.089873,134.0,0.0,0.267458,-0.356445,0.356445,0.267458,1.230769,2.000000,2.730769,1.376538,924.389107,18.095647,14.627869,16.213866,12.597357,8.866938,9.659936,6.752885,7.607222,4.933988,5.757655,3.402552,3.730596,-2.34,1.351091e+06,17.092841,7.113005,3.464258,160.486499,15.200677,11.511791,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Get rid of IPC and descriptor with all zero values

In [0]:
# Drop IPC!!!!
dataset = dataset.drop(['Ipc'], axis=1)

# Drop empty columns
col = list(dataset.head(1))
for c in col[1:]:
  if dataset[c].sum() == 0:
    dataset = dataset.drop([c], axis=1)

Remove correlated descriptors

In [35]:
# Create correlation matrix
corr_matrix = dataset.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
for col in to_drop:
    dataset.drop(col, axis=1, inplace=True)
    
# Update descriptors
descriptors = list(dataset.head(1))[3:]
dataset[descriptors].head()

,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,FpDensityMorgan2,BalabanJ,BertzCT,HallKierAlpha,Kappa3,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,...,fr_Ndealkylation1,fr_Ndealkylation2,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_imidazole,fr_ketone,fr_lactam,fr_methoxy,fr_morpholine,fr_nitro,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.198481,-0.435597,0.013283,0.623001,384.223,0.311075,-0.350740,1.400000,2.120000,1.606288,782.714558,-2.32,3.853433,15.200677,10.847038,0.0,5.817863,5.90718,5.687386,14.908855,4.983979,0.000000,0.0,23.201880,24.974377,31.394564,9.945944,9.717848,40.614309,0.000000,15.284746,0.0,18.883484,17.989423,50.379934,0.000000,10.216698,11.505249,0.0,23.201880,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.517532,-0.411106,0.011515,0.550709,436.310,0.311075,-0.353241,1.444444,2.148148,1.607439,843.272683,-2.42,4.292290,15.200677,5.693928,0.0,5.817863,5.90718,5.687386,14.908855,4.983979,0.000000,0.0,13.847474,46.822227,41.561212,4.923311,9.717848,33.342373,0.000000,15.284746,0.0,38.648865,17.989423,50.370839,0.000000,10.216698,11.505249,0.0,15.929944,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.472623,-0.518193,0.024077,0.644188,474.359,0.339176,-0.462356,1.366667,2.066667,1.707098,983.301265,-2.86,4.648309,19.937540,23.144464,0.0,0.000000,5.90718,5.969305,4.794537,9.778516,5.261892,0.0,0.000000,61.675533,34.991929,12.170333,14.325937,33.624292,5.261892,15.284746,0.0,39.654696,24.596305,44.814141,6.069221,10.216698,5.817863,0.0,15.929944,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.298197,-0.085370,0.085370,0.847033,388.269,0.267458,-0.355369,1.500000,2.250000,1.591350,767.018857,-2.33,3.436063,15.200677,17.581012,0.0,0.000000,5.90718,0.000000,4.794537,4.983979,5.261892,0.0,0.000000,53.894426,35.494845,5.563451,4.794537,27.654986,5.261892,15.284746,0.0,25.807221,17.989423,45.819972,6.069221,10.216698,5.817863,0.0,15.929944,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.387258,-0.040667,0.040667,0.681761,413.319,0.267458,-0.356445,1.230769,2.000000,1.376538,924.389107,-2.34,3.464258,15.200677,11.511791,0.0,0.000000,5.90718,0.000000,4.794537,4.983979,0.000000,0.0,18.199101,59.960793,34.684225,5.516701,4.794537,38.557911,0.000000,15.284746,0.0,25.807221,17.989423,58.325145,0.000000,10.216698,5.817863,0.0,15.929944,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


For now ignore the qualifiers

In [38]:
# comment this latter
y = dataset['SAU Gyr IC50 (µM)'].str.replace('>', '', regex=True)
y = y.str.replace('<', '', regex=True)
#---------------------
dataset['SAU Gyr IC50 (µM)'] = y

# save in Google Drive
dataset.to_csv('/content/drive/My Drive/Hack 7/Gyrase/AZ_clean_test.csv')
dataset.head()

,ID,Smiles,SAU Gyr IC50 (µM),MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,FpDensityMorgan2,BalabanJ,BertzCT,HallKierAlpha,Kappa3,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,...,fr_Ndealkylation1,fr_Ndealkylation2,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_imidazole,fr_ketone,fr_lactam,fr_methoxy,fr_morpholine,fr_nitro,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,AZ1001,c1cc(c(nc1)N2CCC(CC2)NC(=O)c3cc(c([nH]3)Cl)Cl)[N+](=O)[O-],400,12.198481,-0.435597,0.013283,0.623001,384.223,0.311075,-0.350740,1.400000,2.120000,1.606288,782.714558,-2.32,3.853433,15.200677,10.847038,0.0,5.817863,5.90718,5.687386,14.908855,4.983979,0.000000,0.0,23.201880,24.974377,31.394564,9.945944,9.717848,40.614309,0.000000,15.284746,0.0,18.883484,17.989423,50.379934,0.000000,10.216698,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AZ1002,CC(C)c1c(cc([nH]1)C(=O)NC2CCN(CC2)c3c(cccn3)[N+](=O)[O-])Br,50,12.517532,-0.411106,0.011515,0.550709,436.310,0.311075,-0.353241,1.444444,2.148148,1.607439,843.272683,-2.42,4.292290,15.200677,5.693928,0.0,5.817863,5.90718,5.687386,14.908855,4.983979,0.000000,0.0,13.847474,46.822227,41.561212,4.923311,9.717848,33.342373,0.000000,15.284746,0.0,38.648865,17.989423,50.370839,0.000000,10.216698,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AZ1003,CCOC(=O)c1cc(nc(c1C#N)N2CCC(CC2)NC(=O)c3cc(c([nH]3)C)Br)C,7.14,12.472623,-0.518193,0.024077,0.644188,474.359,0.339176,-0.462356,1.366667,2.066667,1.707098,983.301265,-2.86,4.648309,19.937540,23.144464,0.0,0.000000,5.90718,5.969305,4.794537,9.778516,5.261892,0.0,0.000000,61.675533,34.991929,12.170333,14.325937,33.624292,5.261892,15.284746,0.0,39.654696,24.596305,44.814141,6.069221,10.216698,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AZ1004,Cc1c(cc([nH]1)C(=O)NC2CCN(CC2)c3c(cccn3)C#N)Br,9.93,12.298197,-0.085370,0.085370,0.847033,388.269,0.267458,-0.355369,1.500000,2.250000,1.591350,767.018857,-2.33,3.436063,15.200677,17.581012,0.0,0.000000,5.90718,0.000000,4.794537,4.983979,5.261892,0.0,0.000000,53.894426,35.494845,5.563451,4.794537,27.654986,5.261892,15.284746,0.0,25.807221,17.989423,45.819972,6.069221,10.216698,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AZ1005,Cc1c(cc([nH]1)C(=O)NC2CCN(CC2)c3ccc4ccccc4n3)Br,3.31,12.387258,-0.040667,0.040667,0.681761,413.319,0.267458,-0.356445,1.230769,2.000000,1.376538,924.389107,-2.34,3.464258,15.200677,11.511791,0.0,0.000000,5.90718,0.000000,4.794537,4.983979,0.000000,0.0,18.199101,59.960793,34.684225,5.516701,4.794537,38.557911,0.000000,15.284746,0.0,25.807221,17.989423,58.325145,0.000000,10.216698,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Convert to deepchem dataset

In [39]:
# Load dataset
dataset_file = "/content/drive/My Drive/Hack 7/Gyrase/AZ_clean_test.csv"

# Features are already calculated
user_specified_features = descriptors
featurizer = dc.feat.UserDefinedFeaturizer(user_specified_features)

# Loader
loader = dc.data.UserCSVLoader(tasks=["SAU Gyr IC50 (µM)"], 
                               smiles_field="Smiles", 
                               id_field="ID", 
                               featurizer=featurizer,
                               verbose=True)

dc_set = loader.featurize(dataset_file)

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /content/drive/My Drive/Hack 7/Gyrase/AZ_clean_test.csv
Loading shard 1 of size 8192.
TIMING: user specified processing took 0.046 s
TIMING: featurizing shard 0 took 0.063 s
TIMING: dataset construction took 0.132 s
Loading dataset from disk.


/usr/local/lib/python3.6/site-packages/deepchem/data/data_loader.py:126: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X_shard = df.as_matrix(columns=featurizer.feature_fields)


Separate the provided data into train/test subsets 

In [40]:
splitter = dc.splits.ScaffoldSplitter(dataset_file)
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dc_set)

Computing train/valid/test indices
About to generate scaffolds
Generating scaffold 0/981


RDKit ERROR: [03:31:17] SMILES Parse Error: syntax error for input: 'AZ1001'


ValueError: ignored

Normalize it to have zero-mean and unit-standard-deviation

In [194]:
c = dc.data.datasets(Y.train)

AttributeError: ignored